In [1]:
%pip install -r local_req.txt

Note: you may need to restart the kernel to use updated packages.


In [40]:
import torch
import torchaudio
import json

In [43]:
def pre_process(audio_path):
    waveform, sr = torchaudio.load(audio_path)
    if sr != 16000:
        waveform = torchaudio.transforms.Resample(sr, 16000)(waveform)
    return waveform

In [49]:
def get_labels(pred, k):
    if 'json_dict' not in globals():
        global json_dict
        with open("labels.json", "r") as f:
            json_dict = json.load(f)
    labs = pred.topk(k)[1].tolist()[0]
    probs = pred.topk(k)[0].tolist()[0]
    labels = {}
    for i, lab in enumerate(labs):
        labels[json_dict[str(lab)]] = probs[i]
    return labels

In [42]:
from BEATs import BEATs, BEATsConfig

def predict(audio_path, checkpoint_path):
    checkpoint = torch.load(checkpoint_path)
    cfg = BEATsConfig(checkpoint['cfg'])
    BEATs_model = BEATs(cfg)
    BEATs_model.load_state_dict(checkpoint['model'])
    BEATs_model.eval()

    waveform = pre_process(audio_path)
    probs = BEATs_model.extract_features(waveform, padding_mask=None)[0]
    return pred

In [52]:
pred = predict('baby.wav', 'model.pt')
labels = get_labels(pred, 5)
print(pred.size())
print(labels)

torch.Size([1, 527])
{'Wail, moan': 0.5667264461517334, 'Inside, small room': 0.17232947051525116, 'Crying, sobbing': 0.10261154919862747, 'Speech': 0.08706577867269516, 'Giggle': 0.08247208595275879}
